In [19]:
from openai import OpenAI
import os
from tqdm import tqdm

# Setup DashScope OpenAI-compatible client
client = OpenAI(
    api_key="sk-91d59f4cf5ff44cf9f8280a91dacbb1c",
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
)


query = "What is string theory?"
response = client.embeddings.create(
    model="text-embedding-v3",
    input=query,
    encoding_format="float"
)
query_embedding = response.data[0].embedding  # Extract the embedding

In [20]:
# 1. Imports
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores.base import VectorStore
from langchain.schema import Document
from langchain.chains import RetrievalQA
from typing import List, Optional, Tuple

# 2. Your DashVector client (as you already have)
import dashvector
dashvector_key = "sk-eAO7D10gZP2kzhjeKmiI7b6bOHTmoD6B9CAC6318F11F097B99A92126D810D"
client = dashvector.Client(
    api_key=dashvector_key,
    endpoint="vrs-sg-l0z49hq250003n.dashvector.ap-southeast-1.aliyuncs.com"
)
collection = client.get(name="quickstart")

In [21]:
# Search for top 3 similar documents
results = collection.query(
    vector=query_embedding,
    topk=3
)

# Extract document IDs and metadata (if available)
retrieved_docs = [
    {"id": result.id, "score": result.score} 
    for result in results.output
]
print(retrieved_docs)  # Inspect retrieved document IDs and similarity scores

[{'id': '1', 'score': 0.6353}, {'id': '13', 'score': 0.6807}, {'id': '8', 'score': 0.6823}]


In [24]:
chat = ChatOpenAI(
    openai_api_key="sk-669dc1afbce54ccaaecdd27b320777fd",
    openai_api_base="https://dashscope-intl.aliyuncs.com/compatible-mode/v1",
    model="qwen-turbo"
)

C:\Users\Acer\AppData\Local\Temp\ipykernel_15840\1130115753.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


In [25]:
from langchain.chains import RetrievalQA
from langchain.schema import BaseRetriever

class DashVectorRetriever(BaseRetriever):
    def get_relevant_documents(self, query):
        # Embed query
        response = client.embeddings.create(
            model="text-embedding-v3",
            input=query,
            encoding_format="float"
        )
        embedding = response.data[0].embedding
        
        # Retrieve from DashVector
        results = collection.query(vector=embedding, topk=3)
        return [chunks[int(result.id)] for result in results.output]

# Initialize retriever and QA chain
retriever = DashVectorRetriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

# Run RAG
result = qa_chain.run("What is string theory?")
print(result)

C:\Users\Acer\AppData\Local\Temp\ipykernel_15840\3033982481.py:4: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class DashVectorRetriever(BaseRetriever):
C:\Users\Acer\AppData\Local\Temp\ipykernel_15840\3033982481.py:27: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run("What is string theory?")


AttributeError: 'Client' object has no attribute 'embeddings'